<a href="https://colab.research.google.com/github/luiscaballeroramos/DeCoder/blob/EC3_1_8_DesignOfJoints/DeCoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Preliminary Tasks

Install and import PyPDF2 library to open and read the PDF file containing the code

In [1]:
!pip install PyPDF2
import PyPDF2

Clone the GitHub repository containing the PDF file with code

In [2]:
!git clone -b EC3_1_8_DesignOfJoints https://github.com/luiscaballeroramos/DeCoder.git

fatal: destination path 'DeCoder' already exists and is not an empty directory.


Import libraries used in this tool

In [3]:
import numpy as np
import regex as re
from tabulate import tabulate

# 1. Read PDF

Read the .pdf file with PyPF2 library

In [4]:
pdfFile=open('DeCoder/EC 1993.1.8.2005-1.pdf','rb')
pdfReader=PyPDF2.PdfFileReader(pdfFile)

In [5]:
numPages=pdfReader.numPages
print(numPages)

135


Get the pages content as dictionary (pages) with:
* key = page number (pageNum)
* value = page content as string (pageContent)



In [6]:
pages={}
pageContent=''
for pageNum in range(0,numPages):
  pdfPage=pdfReader.getPage(pageNum)
  pageContent=pdfPage.extractText()
  pages[pageNum]=pageContent
  pass

# 2. Table Of Contents (TOC)
2.0. TOC Definition (settings)

2.1. TOC Location (pages)

2.2. TOC Contents (toc's)

# 2.0. TOC Definition

Set TOC identifiers, keywords that could be used to refering TOC

In [7]:
TOCidentifiers=['contents','Contents',
                 'table of contents','Table of contents','Table Of Contents']

Description of toc elements by regex expressions 

In [8]:
refidRegex='\d'
refsepRegex='\.'
refsep='.'
namRegex='[\s\w]*'
sepRegex='\.{2,}'
numRegex='\d+'

In [9]:
class TOC():
  def __init__(self,refidRegex,refsepRegex,refsep,namRegex,sepRegex,numRegex):
    self.refidRegex=refidRegex
    self.refsepRegex=refsepRegex
    self.refsep=refsep
    self.refRegex=refidRegex+'(?:'+refsepRegex+'+'+refidRegex+'*)+'
    self.namRegex=namRegex
    self.sepRegex=sepRegex
    self.numRegex=numRegex
    self.tocRegex=self.refRegex+'\s*'+namRegex+'\s*'+sepRegex+'\s*'+numRegex
    self.tocList=[]
    pass
  def addToc(self,toc):
    if len(list(x for x in self.tocList if x.ref == toc.ref)) == 0:
      self.tocList.append(toc)
      pass
    pass
  def printTOC(self,num=None):
    table=[]
    if num is None:
      for toc in TOC_EC3.tocList:
        table.append([toc.ref,toc.name,toc.begin.page,toc.level])
        pass
    else:
      toc=TOC_EC3.tocList[num]
      table.append([toc.ref,toc.name,toc.begin.page,toc.level])
      pass
    print(tabulate(table,headers=['Ref.','Name','Begin page','Level'], tablefmt='orgtbl'))
    pass
  pass
TOC_EC3=TOC(refidRegex,refsepRegex,refsep,namRegex,sepRegex,numRegex)

In [10]:
class PDFlocation():
  def __init__(self,page,begin=None,end=None):
    self.page=page
    if begin is None:
      self.begin=None
      self.end =None
    else:
      self.begin=int(begin)
      if end is None:
        self.end=int(begin)
      else:
        self.end=end
        pass
      pass
    pass
  pass

In [11]:
class toc():
  def __init__(self,ref,name,begin=None,end=None,level=None):
    self.ref=ref
    self.name=name
    self.begin=begin
    self.end=end
    self.setLevel()
    pass
  def setLevel(self):
    self.level = len(list(filter(lambda item: item, re.split('\.*',self.ref))))-1
    pass
  pass

# 2.1. TOC Location

TOC identifiers apearances in each document page 

In [12]:
TOCcoincidences={}
TOCtotalCoincidences={}
for pageNum, pageContent in pages.items():
  aux=[]
  for TOCidentifier in TOCidentifiers:
    aux.append(pageContent.count(TOCidentifier))
    pass
  TOCcoincidences[pageNum]=aux
  TOCtotalCoincidences[pageNum]=sum(aux)
  pass

Total of TOC identifiers apearances in PDF document

In [14]:
sumTOCTotalCoincidences=sum(TOCtotalCoincidences.values())
print('Number of TOC indentifiers: {0}'.format(sumTOCTotalCoincidences))

Number of TOC indentifiers: 1


Max TOC identifiers apearances in one page and the page where this happens

In [18]:
maxTOCTotalCoincidences=max(TOCtotalCoincidences.values())
maxTOCTotalCoincidencesPages = [key  for (key, value) in TOCtotalCoincidences.items() if value == maxTOCTotalCoincidences]
print('Number of Max TOC indentifiers in One Page: {0}'.format(maxTOCTotalCoincidences))
print('Pages with Max TOC indentifiers in One Page: {0}'.format(maxTOCTotalCoincidencesPages))

Number of Max TOC indentifiers in One Page: 1
Pages with Max TOC indentifiers in One Page: [3]


Pages where toc pattern (Regex) is detected

In [20]:
tocCoincidences={}
for pageNum, pageContent in pages.items():
  tocCoincidences[pageNum]=len(re.findall(TOC_EC3.tocRegex,pages[pageNum]))
  pass

In [21]:
noNulltocCoincidencesPages = [key  for (key, value) in tocCoincidences.items() if value != 0]
print('Pages with toc Regex: {0}'.format(noNulltocCoincidencesPages))

Pages with toc Regex: [3, 4, 5]


In [22]:
TOCpages=noNulltocCoincidencesPages

# 2.2. TOC Contents

Look for complete tocRegex in TOCpages

In [23]:
pattern = re.compile(TOC_EC3.tocRegex)
for page in TOCpages:
  for match in pattern.finditer(pages[page]):
    [split1,split2]=re.split(TOC_EC3.sepRegex,match.group())
    ref=re.search(TOC_EC3.refRegex,split1).group()
    split1=re.sub(TOC_EC3.refRegex,'',split1)
    nam=re.search(TOC_EC3.namRegex,split1).group()
    split1=re.sub(TOC_EC3.namRegex,'',split1)
    num=re.search(TOC_EC3.numRegex,split2).group()
    beginLocation=PDFlocation(num)
    toc_EC3=toc(ref,nam,beginLocation)
    TOC_EC3.addToc(toc_EC3)
    pass
  pass
TOC_EC3.tocList.sort(key=lambda x: x.ref, reverse=False)
TOC_EC3.printTOC()

| Ref.   | Name                                                                           |   Begin page |   Level |
|--------+--------------------------------------------------------------------------------+--------------+---------|
| 1.3    | Distinction between Principles and Application Rules                           |            1 |       1 |
| 1.5    | SY111bols                                                                      |           13 |       1 |
| 2.1    | AsslIlnptions                                                                  |           18 |       1 |
| 2.2    | General requirenlents                                                          |           18 |       1 |
| 2.4    | Resistance of joints                                                           |           18 |       1 |
| 2.5    | Design assunlptions                                                            |           19 |       1 |
| 2.7    | Eccentricity at intersections                        

Complete with references that can be deduced to exist in TOC

In [38]:
TOC_EC3.printTOC(11)
toc=TOC_EC3.tocList[11]
levels=re.split(TOC_EC3.refsepRegex,toc.ref)
levels=list(map(int,levels))
print(levels)
refLeft=[]
for level in range(toc.level+1):
  numLevel=levels[level]
  #if numLevel=1 there is no previous ref in this level
  if numLevel!=1:
    #find the first part of the ref at the levels>0
    if level!=0:
      print(level)
      prevref=''
      for i in range(level):
        prevref=prevref+str(levels[i])+TOC_EC3.refsep
        pass
    else:
      prevref=''
      pass
    for i in range(1,numLevel+1):
      refLeft.append(prevref+str(i)+TOC_EC3.refsep)
      pass
    pass
  pass
print(refLeft)

| Ref.   | Name                                                                           |   Begin page |   Level |
|--------+--------------------------------------------------------------------------------+--------------+---------|
| 3.10.3 | Angles connected by one and other unsymmetrically connected members in tension |           33 |       2 |
[3, 10, 3]
1
2
['1.', '2.', '3.', '3.1.', '3.2.', '3.3.', '3.4.', '3.5.', '3.6.', '3.7.', '3.8.', '3.9.', '3.10.', '3.10.1.', '3.10.2.', '3.10.3.']


# Close PDF

In [ ]:
pdfFile.close()